In [1]:
data_var = '2024-02-10'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
14854,2024-02-10,Eua Ncaa,15:00,Gardner Webb,High Point,2.33,1.65,155.5,1.80,1.95,1.5,2.10,1.67,4vjYxXQe,0.429185,0.606061,0.555556,0.512821,0.035245,140.140,43.756137,0.312232,2.4,1.341641,0.559017,108.75,1.818,0.567424,0.312114,27.0,122.378,28.429840,0.232312,2.4,1.341641,0.559017,132.84,1.478,0.315071,0.213174,35.0,75,81,1.45,1.64,110.098,134.458,0.241624,0.056569,0.161303,-0.10,-0.020,-66.500000,0.000000,0.0,0.000000,1.49,0.298,2.181208,0.514446,0.5,-0.014446
14855,2024-02-10,Eua Ncaa,16:00,Binghamton,Bryant University,2.47,1.58,148.5,1.91,1.91,2.5,2.00,1.71,pzVLW2rK,0.404858,0.632911,0.523560,0.523560,0.037770,229.342,168.598459,0.735140,1.8,1.643168,0.912871,116.61,4.050,3.724184,0.919552,3.0,114.732,17.127591,0.149283,1.8,1.643168,0.912871,124.95,1.556,0.450422,0.289474,21.0,69,85,1.69,1.47,139.730,133.670,0.310778,0.000000,0.110545,-0.60,-0.120,-12.250000,0.000000,0.0,0.000000,1.40,0.280,2.071429,0.518123,0.6,0.081877
14856,2024-02-10,Eua Ncaa,15:30,Youngstown State,Wisc. Green Bay,1.21,4.67,139.5,1.80,1.95,-10.5,2.00,1.71,rLL2HdyU,0.826446,0.214133,0.555556,0.512821,0.040579,128.288,42.765480,0.333355,3.0,0.000000,0.000000,119.31,1.430,0.480573,0.336065,122.0,125.928,35.795155,0.284251,2.4,1.341641,0.559017,153.90,1.842,0.769103,0.417537,30.0,97,81,1.23,1.90,123.180,209.774,0.832173,0.056569,0.110545,2.17,0.434,0.483871,0.701446,0.9,0.198554,2.88,0.576,6.371528,0.371557,0.5,0.128443
14857,2024-02-10,Eua Ncaa,15:00,VMI,Samford,11.50,1.05,170.5,1.80,1.95,16.5,2.10,1.67,M5XgJI6H,0.086957,0.952381,0.555556,0.512821,0.039337,760.762,384.747467,0.505740,0.0,0.000000,NaN,735.00,9.904,4.895287,0.494274,-57.0,130.254,28.955163,0.222298,2.4,1.341641,0.559017,148.98,1.726,0.421224,0.244047,26.0,84,78,8.75,1.91,515.842,140.436,1.177572,0.056569,0.161303,-0.62,-0.124,-84.677419,0.000000,0.0,0.000000,2.30,0.460,0.108696,0.542552,0.7,0.157448
14858,2024-02-10,Eua Ncaa,15:00,UMBC Retrievers,Vermont,4.91,1.19,147.5,1.80,1.95,9.5,2.00,1.71,URnZ8ajD,0.203666,0.840336,0.555556,0.512821,0.044002,278.710,166.420766,0.597111,0.6,1.341641,2.236068,290.40,3.180,1.510166,0.474895,-12.0,98.148,33.344533,0.339737,2.4,1.341641,0.559017,70.76,1.422,0.423993,0.298167,50.0,80,61,3.63,1.16,179.436,109.872,0.862438,0.056569,0.110545,-2.62,-0.524,-7.461832,0.464389,0.4,-0.064389,1.60,0.320,0.593750,0.561937,0.6,0.038063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15172,2024-02-10,Europa Liga Aba,13:00,Cedevita Olimpija,FMP,1.33,3.12,166.5,1.86,1.80,-8.5,2.00,1.72,rR2KBuOI,0.751880,0.320513,0.537634,0.555556,0.072393,179.020,107.934184,0.602917,1.8,1.643168,0.912871,109.52,2.004,1.044165,0.521040,39.0,239.316,188.737349,0.788653,0.6,1.341641,2.236068,131.82,3.872,3.775483,0.975073,-50.0,74,78,1.48,1.69,166.218,257.220,0.568863,0.023184,0.106446,2.67,0.534,0.617978,0.000000,0.0,0.000000,-1.22,-0.244,-8.688525,0.000000,0.0,0.000000
15173,2024-02-10,Eua Ncaa,22:30,Missouri,Mississippi St.,2.74,1.45,139.5,1.80,1.91,5.5,1.83,1.83,Us3ztVye,0.364964,0.689655,0.555556,0.523560,0.054619,191.518,107.384711,0.560703,0.0,0.000000,NaN,148.20,3.088,2.04167

## Apostas do dia

### Back Home V3 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
14870,16:00,Eua Ncaa,Bucknell,Boston University,1.58,Back Home
14871,16:00,Eua Ncaa,North Dakota St.,Nebraska O.,1.67,Back Home
14873,17:00,Eua Ncaa,Jacksonville,North Alabama,2.00,Back Home
14880,16:00,Eua Ncaa,Robert Morris,Wisc. Milwaukee,2.05,Back Home
14907,14:00,República Tcheca Nbl,Brno,Opava,2.45,Back Home
14927,12:00,Suécia Superettan,Ockelbo,Huddinge,2.50,Back Home
14944,19:30,Eua Ncaa,California,UCLA,2.00,Back Home
14983,18:00,Eua Ncaa,Cal St. Bakersfield,CS Fullerton,1.87,Back Home
14990,17:00,Eua Ncaa,South Carolina St,Chicago St,2.10,Back Home
15001,18:30,Eua Ncaa,Houston Christian,Incarnate Word,2.10,Back Home
